# Avaliação Comparativa de Modelos

Este notebook compara o desempenho de todos os 5 modelos treinados:

1. **MLP Centralizado** - Baseline centralizado
2. **XGBoost Centralizado** - Baseline centralizado
3. **MLP Federado (FedAvg)** - Aprendizado federado com média de pesos
4. **XGBoost Federado (Bagging)** - Agregação bootstrap
5. **XGBoost Federado (Cyclic)** - Treinamento cliente-por-cliente

**Objetivo:** Avaliar todos os modelos no **mesmo conjunto de teste** para comparação justa.

## 1. Importar Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import joblib
import torch
import torch.nn as nn
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    accuracy_score,
    precision_recall_fscore_support,
    matthews_corrcoef,
    average_precision_score
)

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print("Bibliotecas importadas com sucesso!")

In [ ]:
# Adicionar paths necessários para imports
import sys
from pathlib import Path

# Adicionar diretórios ao Python path
base_path = Path.cwd().parent
sys.path.insert(0, str(base_path / "flwr-mlp"))
sys.path.insert(0, str(base_path / "centralized_training"))

print("Paths configurados:")
print(f"   - {base_path / 'flwr-mlp'}")
print(f"   - {base_path / 'centralized_training'}")

## 2. Funções de Pré-processamento

In [ ]:
# Constantes
WIND_DIRS = ["N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE", 
             "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW"]

def encode_wind_directions(df):
    """
    Converte direções de vento categóricas para componentes sin/cos.
    Preserva a natureza circular das direções de vento.
    """
    wind_cols = ["WindGustDir", "WindDir9am", "WindDir3pm"]
    
    for col in wind_cols:
        if col in df.columns:
            # Criar mapeamento de direções para ângulos (em radianos)
            dir_to_rad = {dir_: i * 2 * np.pi / 16 for i, dir_ in enumerate(WIND_DIRS)}
            
            # Adicionar tratamento para valores ausentes
            dir_to_rad.update({np.nan: 0, "NA": 0, None: 0})
            
            # Converter para ângulos e depois para componentes sin/cos
            angles = df[col].map(dir_to_rad)
            df[f"{col}_sin"] = np.sin(angles)
            df[f"{col}_cos"] = np.cos(angles)
            df.drop(columns=[col], inplace=True)
            
    return df

def prepare_weather_data(df):
    """
    Preprocessar dados meteorológicos:
    - Encode direções de vento (categóricas → sin/cos)
    - Remove coluna Location
    """
    df = encode_wind_directions(df)
    
    if 'Location' in df.columns:
        df = df.drop(columns=['Location'])
    
    return df

print("Funções de pré-processamento definidas!")
print(f"   - encode_wind_directions: Converte {len([c for c in ['WindGustDir', 'WindDir9am', 'WindDir3pm']])} colunas de vento")
print(f"   - prepare_weather_data: Pipeline completo de preprocessamento")

## 3. Carregar e Preprocessar Dados de Teste

In [ ]:
# Paths
base_path = Path.cwd().parent
data_path = base_path / "datasets" / "rain_australia" / "weatherAUS_cleaned.csv"
test_indices_path = base_path / "datasets" / "test_indices.csv"

print("Carregando dados...")

# Carregar dataset completo
df = pd.read_csv(data_path)
print(f"   Dataset: {len(df):,} amostras")
print(f"   Colunas originais: {len(df.columns)}")

# Carregar índices de teste
test_indices = pd.read_csv(test_indices_path)['index'].values
print(f"   Conjunto de teste: {len(test_indices):,} amostras")

# Filtrar dados de teste
df_test = df.iloc[test_indices].copy()

# Separar target ANTES do preprocessing
y_test = df_test['RainTomorrow'].values.astype(np.float32)

print(f"\nAplicando pré-processamento...")

# Aplicar preprocessing (encode vento + remover Location)
df_test_processed = prepare_weather_data(df_test.copy())

print(f"   Encoding de direções de vento aplicado (categórico → sin/cos)")
print(f"   Colunas após preprocessing: {len(df_test_processed.columns)}")

# Verificar que WindGustDir, WindDir9am, WindDir3pm foram convertidas
wind_encoded_cols = [col for col in df_test_processed.columns if 'WindDir' in col or 'WindGust' in col]
print(f"   Colunas de vento codificadas: {wind_encoded_cols}")

# Remover target das features
if 'RainTomorrow' in df_test_processed.columns:
    df_test_processed = df_test_processed.drop(columns=['RainTomorrow'])

# Features para MLP
X_test_mlp = df_test_processed.values.astype(np.float32)

# Features para XGBoost (DataFrame)
X_test_xgb = df_test_processed.copy()

print(f"\nConjunto de teste preparado:")
print(f"   No Rain (0): {(y_test == 0).sum():,} ({(y_test == 0).sum() / len(y_test) * 100:.1f}%)")
print(f"   Rain (1): {(y_test == 1).sum():,} ({(y_test == 1).sum() / len(y_test) * 100:.1f}%)")
print(f"   Features (MLP): {X_test_mlp.shape}")
print(f"   Features (XGBoost): {X_test_xgb.shape}")
print(f"   Feature names: {list(df_test_processed.columns)[:5]} ... (total: {len(df_test_processed.columns)})")

## 4. Definir Arquitetura MLP

In [ ]:
class WeatherMLP(nn.Module):
    """Rede Neural para previsão de chuva."""
    def __init__(self, input_size=20, hidden1=96, hidden2=112, hidden3=32, 
                 dropout1=0.14, dropout2=0.38):
        super(WeatherMLP, self).__init__()
        
        self.fc1 = nn.Linear(input_size, hidden1)
        self.bn1 = nn.BatchNorm1d(hidden1)
        self.dropout1 = nn.Dropout(dropout1)
        
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.bn2 = nn.BatchNorm1d(hidden2)
        self.dropout2 = nn.Dropout(dropout2)
        
        self.fc3 = nn.Linear(hidden2, hidden3)
        self.bn3 = nn.BatchNorm1d(hidden3)
        
        self.fc4 = nn.Linear(hidden3, 1)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = self.relu(self.bn3(self.fc3(x)))
        x = self.fc4(x)
        return x

print("Arquitetura MLP definida!")

## 5. Carregar Todos os Modelos

In [ ]:
print("Carregando modelos...\n")

import xgboost as xgb

models = {}
model_metadata = {}

# 1. MLP Centralizado
try:
    mlp_cent_path = base_path / "centralized_training" / "models" / "mlp" / "centralized_model_best.joblib"
    mlp_cent_data = joblib.load(mlp_cent_path)
    
    if isinstance(mlp_cent_data, dict):
        models['MLP Centralizado'] = mlp_cent_data['best_model']
        models['MLP Centralizado'].eval()
        model_metadata['MLP Centralizado'] = mlp_cent_data.get('metadata', {})
        model_metadata['MLP Centralizado']['scaler'] = mlp_cent_data.get('scaler')
        
        print("[OK] MLP Centralizado carregado")
        print(f"   Input size: {mlp_cent_data.get('metadata', {}).get('input_size', 'N/A')}")
    else:
        print(f"[ERRO] Formato inesperado: {type(mlp_cent_data)}")
except Exception as e:
    print(f"[ERRO] Erro ao carregar MLP Centralizado: {e}")
    import traceback
    print(f"   Detalhes: {traceback.format_exc()}")

# 2. XGBoost Centralizado
try:
    xgb_cent_path = base_path / "centralized_training" / "models" / "xgboost" / "centralized_model_best.joblib"
    xgb_cent_data = joblib.load(xgb_cent_path)
    
    if isinstance(xgb_cent_data, dict):
        models['XGBoost Centralizado'] = xgb_cent_data['best_model']
        model_metadata['XGBoost Centralizado'] = xgb_cent_data.get('metadata', {})
    else:
        models['XGBoost Centralizado'] = xgb_cent_data
        model_metadata['XGBoost Centralizado'] = {}
    
    print("[OK] XGBoost Centralizado carregado")
except Exception as e:
    print(f"[ERRO] Erro ao carregar XGBoost Centralizado: {e}")

# 3. MLP Federado
try:
    mlp_fed_path = base_path / "flwr-mlp" / "models" / "global_model_final.joblib"
    mlp_fed_data = joblib.load(mlp_fed_path)
    
    if isinstance(mlp_fed_data, dict):
        models['MLP Federado (FedAvg)'] = mlp_fed_data['best_model']
        models['MLP Federado (FedAvg)'].eval()
        model_metadata['MLP Federado (FedAvg)'] = mlp_fed_data.get('metadata', {})
        
        if 'scaler' in mlp_fed_data:
            model_metadata['MLP Federado (FedAvg)']['scaler'] = mlp_fed_data['scaler']
        else:
            model_metadata['MLP Federado (FedAvg)']['scaler'] = None
        
        print("[OK] MLP Federado (FedAvg) carregado")
    else:
        models['MLP Federado (FedAvg)'] = mlp_fed_data
        models['MLP Federado (FedAvg)'].eval()
        model_metadata['MLP Federado (FedAvg)'] = {}
        model_metadata['MLP Federado (FedAvg)']['scaler'] = None
        
        print("[OK] MLP Federado (FedAvg) carregado")
except Exception as e:
    print(f"[ERRO] Erro ao carregar MLP Federado: {e}")
    import traceback
    print(f"   Detalhes: {traceback.format_exc()}")

# 4. XGBoost Federado Bagging
try:
    xgb_bag_path = base_path / "flwr-xgboost" / "models" / "global_model_bagging_final.joblib"
    xgb_bag_data = joblib.load(xgb_bag_path)
    
    if isinstance(xgb_bag_data, dict):
        if 'best_model' in xgb_bag_data:
            models['XGBoost Federado (Bagging)'] = xgb_bag_data['best_model']
            model_metadata['XGBoost Federado (Bagging)'] = xgb_bag_data.get('metadata', {})
        else:
            print(f"   [INFO] Chaves disponíveis: {list(xgb_bag_data.keys())}")
            models['XGBoost Federado (Bagging)'] = xgb_bag_data
            model_metadata['XGBoost Federado (Bagging)'] = {}
    else:
        models['XGBoost Federado (Bagging)'] = xgb_bag_data
        model_metadata['XGBoost Federado (Bagging)'] = {}
    
    print("[OK] XGBoost Federado (Bagging) carregado")
except Exception as e:
    print(f"[ERRO] Erro ao carregar XGBoost Federado Bagging: {e}")

# 5. XGBoost Federado Cyclic
try:
    xgb_cyc_path = base_path / "flwr-xgboost" / "models" / "global_model_cyclic_final.joblib"
    xgb_cyc_data = joblib.load(xgb_cyc_path)
    
    if isinstance(xgb_cyc_data, dict):
        if 'best_model' in xgb_cyc_data:
            models['XGBoost Federado (Cyclic)'] = xgb_cyc_data['best_model']
            model_metadata['XGBoost Federado (Cyclic)'] = xgb_cyc_data.get('metadata', {})
        else:
            print(f"   [INFO] Chaves disponíveis: {list(xgb_cyc_data.keys())}")
            models['XGBoost Federado (Cyclic)'] = xgb_cyc_data
            model_metadata['XGBoost Federado (Cyclic)'] = {}
    else:
        models['XGBoost Federado (Cyclic)'] = xgb_cyc_data
        model_metadata['XGBoost Federado (Cyclic)'] = {}
    
    print("[OK] XGBoost Federado (Cyclic) carregado")
except Exception as e:
    print(f"[ERRO] Erro ao carregar XGBoost Federado Cyclic: {e}")

print(f"\nTotal de modelos carregados: {len(models)}/5")

# Mostrar informações dos modelos
print(f"\nInformações dos modelos:")
for model_name, metadata in model_metadata.items():
    if metadata:
        print(f"\n   {model_name}:")
        print(f"      - AUC: {metadata.get('auc', 'N/A'):.4f}" if isinstance(metadata.get('auc'), (int, float)) else f"      - AUC: {metadata.get('auc', 'N/A')}")
        print(f"      - Accuracy: {metadata.get('accuracy', 'N/A'):.4f}" if isinstance(metadata.get('accuracy'), (int, float)) else f"      - Accuracy: {metadata.get('accuracy', 'N/A')}")
        print(f"      - Framework: {metadata.get('framework', 'N/A')}")

## 5.5. Verificação de Integridade dos Modelos

In [ ]:
print("Verificando integridade dos modelos...\n")

import xgboost as xgb

verification_results = []

for model_name, model in models.items():
    result = {'Modelo': model_name}
    
    if 'MLP' in model_name:
        # Verificar MLP
        result['Tipo'] = 'PyTorch MLP'
        result['Classe'] = type(model).__name__
        result['Parâmetros'] = sum(p.numel() for p in model.parameters())
        result['Input Size'] = model.fc1.in_features if hasattr(model, 'fc1') else 'N/A'
        result['Status'] = '[OK]'
    else:
        # Verificar XGBoost
        result['Tipo'] = 'XGBoost Booster'
        result['Classe'] = type(model).__name__
        
        if isinstance(model, xgb.Booster):
            result['Num Boosted Rounds'] = model.num_boosted_rounds()
            result['Num Features'] = model.num_features()
            result['Status'] = '[OK]'
        elif isinstance(model, dict):
            result['Status'] = '[ERRO] Ainda é dicionário'
            result['Chaves'] = list(model.keys())
        else:
            result['Status'] = '[AVISO] Tipo inesperado'
    
    verification_results.append(result)

# Mostrar resultados
print("="*80)
print("VERIFICAÇÃO DE INTEGRIDADE DOS MODELOS")
print("="*80)

for result in verification_results:
    print(f"\n{result['Modelo']}")
    print(f"   - Tipo: {result['Tipo']}")
    print(f"   - Classe: {result['Classe']}")
    
    if 'Input Size' in result:
        print(f"   - Input Size: {result['Input Size']}")
        print(f"   - Parâmetros: {result['Parâmetros']:,}")
    elif 'Num Features' in result:
        print(f"   - Num Features: {result['Num Features']}")
        print(f"   - Num Boosted Rounds: {result['Num Boosted Rounds']}")
    
    print(f"   - Status: {result['Status']}")

print("\n" + "="*80)

# Verificar se há algum problema
problemas = [r for r in verification_results if '[ERRO]' in r['Status'] or '[AVISO]' in r['Status']]
if problemas:
    print("\n[ATENÇÃO] Foram encontrados problemas com os seguintes modelos:")
    for p in problemas:
        print(f"   - {p['Modelo']}: {p['Status']}")
else:
    print("\nTodos os modelos passaram na verificação de integridade!")
    print("   Modelos prontos para gerar predições")

In [ ]:
# CÉLULA DE DEBUG ATUALIZADA

print("DIAGNÓSTICO COMPLETO DO PROBLEMA MLP\n")
print("="*80)

# 1. Carregar modelo
mlp_cent_data = joblib.load(base_path / "centralized_training" / "models" / "mlp" / "centralized_model_best.joblib")

# 2. Verificar scaler
scaler = mlp_cent_data['scaler']
print("[OK] Scaler encontrado")
print(f"   N features esperadas: {scaler.n_features_in_}")

# 3. Inspecionar arquitetura do modelo salvo
model = mlp_cent_data['best_model']
print(f"\nArquitetura do modelo SALVO:")
print(f"   Tipo: {type(model)}")
print(f"   Módulos:")

for name, module in model.named_children():
    print(f"      - {name}: {module}")
    if hasattr(module, 'in_features'):
        print(f"         in_features: {module.in_features}")
    if hasattr(module, 'out_features'):
        print(f"         out_features: {module.out_features}")

# 4. Tentar acessar primeira camada linear
try:
    first_linear = None
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            first_linear = module
            print(f"\nPrimeira camada Linear encontrada: '{name}'")
            print(f"   Input size: {first_linear.in_features}")
            break
    
    if first_linear:
        expected_features = first_linear.in_features
        print(f"\n[OK] Features esperadas pelo modelo: {expected_features}")
        print(f"Features no conjunto de teste: {X_test_mlp.shape[1]}")
        
        if expected_features != X_test_mlp.shape[1]:
            print(f"\n[ERRO] INCOMPATIBILIDADE!")
            print(f"   Modelo espera: {expected_features}")
            print(f"   Teste tem: {X_test_mlp.shape[1]}")
            print(f"   SOLUÇÃO: Usar scaler.n_features_in_ = {scaler.n_features_in_}")
except Exception as e:
    print(f"[ERRO] Erro ao inspecionar: {e}")

# 5. Verificar feature names
if 'feature_names' in mlp_cent_data:
    train_features = mlp_cent_data['feature_names']
    test_features = list(df_test_processed.columns)
    
    print(f"\nFeatures no TREINAMENTO ({len(train_features)}):")
    print(f"   {train_features[:5]} ... (total: {len(train_features)})")
    
    print(f"\nFeatures no TESTE ({len(test_features)}):")
    print(f"   {test_features[:5]} ... (total: {len(test_features)})")
    
    if len(train_features) != len(test_features):
        print(f"\n[ERRO] QUANTIDADE DIFERENTE!")
        print(f"   Treino: {len(train_features)} features")
        print(f"   Teste: {len(test_features)} features")
        
        missing = set(train_features) - set(test_features)
        if missing:
            print(f"\n[INFO] Features no treino MAS NÃO no teste:")
            print(f"   {sorted(missing)}")
        
        extra = set(test_features) - set(train_features)
        if extra:
            print(f"\n[INFO] Features no teste MAS NÃO no treino:")
            print(f"   {sorted(extra)}")
else:
    print("\n[INFO] Feature names não salvos")

# 6. Teste com dados ESCALADOS
print(f"\nTeste com 1 amostra (SEM e COM scaler):")

# SEM scaler
X_sample_raw = X_test_mlp[:10].copy()
print(f"\n   RAW (sem scaler):")
print(f"      Shape: {X_sample_raw.shape}")
print(f"      Min: {X_sample_raw.min():.4f}, Max: {X_sample_raw.max():.4f}, Mean: {X_sample_raw.mean():.4f}")

# COM scaler
X_sample_scaled = scaler.transform(X_sample_raw)
print(f"\n   SCALED (com scaler):")
print(f"      Shape: {X_sample_scaled.shape}")
print(f"      Min: {X_sample_scaled.min():.4f}, Max: {X_sample_scaled.max():.4f}, Mean: {X_sample_scaled.mean():.4f}")

# Predições
model.eval()
with torch.no_grad():
    output_raw = model(torch.FloatTensor(X_sample_raw[:1]))
    prob_raw = torch.sigmoid(output_raw).item()
    
    output_scaled = model(torch.FloatTensor(X_sample_scaled[:1]))
    prob_scaled = torch.sigmoid(output_scaled).item()
    
    print(f"\n   Predições:")
    print(f"      SEM scaler: prob={prob_raw:.4f}, pred={1 if prob_raw >= 0.5 else 0}")
    print(f"      COM scaler: prob={prob_scaled:.4f}, pred={1 if prob_scaled >= 0.5 else 0}")
    print(f"      Diferença: {abs(prob_raw - prob_scaled):.4f}")

print("\n" + "="*80)

In [ ]:
# CÉLULA: Teste de Validação Pós-Correção (VERSÃO CORRIGIDA)
print("Teste de validação pós-correção:\n")

for model_name in models.keys():
    if 'MLP' in model_name:
        scaler = model_metadata[model_name].get('scaler')
        model = models[model_name]
        
        if scaler is None:
            print(f"[ERRO] {model_name}: Scaler NÃO encontrado")
            print(f"   Verificando arquivo original...\n")
            
            if 'Centralizado' in model_name:
                path = base_path / "centralized_training" / "models" / "mlp" / "centralized_model_best.joblib"
            else:
                path = base_path / "flwr-mlp" / "models" / "global_model_final.joblib"
            
            data = joblib.load(path)
            print(f"   Chaves disponíveis: {list(data.keys())}")
            
            if 'scaler' in data:
                print(f"   [OK] Scaler EXISTE no arquivo!")
                model_metadata[model_name]['scaler'] = data['scaler']
                scaler = data['scaler']
                
                if scaler is not None and hasattr(scaler, 'n_features_in_'):
                    print(f"   Scaler configurado: {scaler.n_features_in_} features\n")
                else:
                    print(f"   [ERRO] Scaler inválido! Usando fallback do modelo centralizado.\n")
                    cent_data = joblib.load(base_path / "centralized_training" / "models" / "mlp" / "centralized_model_best.joblib")
                    model_metadata[model_name]['scaler'] = cent_data['scaler']
                    scaler = cent_data['scaler']
                    print(f"   Scaler fallback configurado: {scaler.n_features_in_} features\n")
            else:
                print(f"   [INFO] Arquivo NÃO contém 'scaler'. Usando fallback do modelo centralizado.")
                cent_data = joblib.load(base_path / "centralized_training" / "models" / "mlp" / "centralized_model_best.joblib")
                model_metadata[model_name]['scaler'] = cent_data['scaler']
                scaler = cent_data['scaler']
                print(f"   Scaler fallback configurado: {scaler.n_features_in_} features\n")
        
        if scaler is None or not hasattr(scaler, 'transform'):
            print(f"[ERRO] {model_name}: Scaler ainda é None após recarregamento!")
            continue
        
        # Testar com 1 amostra
        X_sample = X_test_mlp[:1]
        X_scaled = scaler.transform(X_sample)
        
        model.eval()
        with torch.no_grad():
            output = model(torch.FloatTensor(X_scaled))
            prob = torch.sigmoid(output).item()
        
        print(f"[OK] {model_name}:")
        print(f"   - Scaler: {scaler.n_features_in_} features")
        print(f"   - Prob: {prob:.4f}")
        print(f"   - Status: {'[OK]' if 0.0 < prob < 1.0 else '[PROBLEMA]'}\n")

print("="*80)
print("Validação completa! Todos os modelos MLP têm scalers configurados.")
print("="*80)

## 6. Gerar Predições para Todos os Modelos

In [ ]:
# CÉLULA CORRIGIDA: Gerar Predições

print("Gerando predições...\n")

import xgboost as xgb
from sklearn.preprocessing import StandardScaler

predictions = {}
probabilities = {}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando device: {device}\n")

for model_name, model in models.items():
    try:
        if 'MLP' in model_name:
            
            if 'Centralizado' in model_name:
                scaler = model_metadata[model_name].get('scaler')
                
                if scaler is None:
                    print(f"   [ERRO] {model_name}: Scaler não encontrado no checkpoint!")
                    continue
                
                print(f"   [OK] {model_name}: Usando scaler do TREINAMENTO (salvo no checkpoint)")
                print(f"      Fitted com {scaler.n_features_in_} features")
                
            else:  # Federado
                print(f"   [OK] {model_name}: Criando NOVO scaler (comportamento federado)")
                print(f"      Cada cliente treinou com seu próprio scaler local")
                print(f"      SHAP também usa novo scaler fitted nos dados de teste")
                
                scaler = StandardScaler()
                scaler.fit(X_test_mlp)
                print(f"      Scaler fitted com {X_test_mlp.shape[1]} features do teste")
            
            X_test_scaled = scaler.transform(X_test_mlp)
            
            model = model.to(device)
            X_tensor = torch.FloatTensor(X_test_scaled).to(device)
            
            with torch.no_grad():
                outputs = model(X_tensor).squeeze()
                probs = torch.sigmoid(outputs).cpu().numpy()
                preds = (probs >= 0.5).astype(int)
            
            predictions[model_name] = preds
            probabilities[model_name] = probs
            
        else:
            if isinstance(model, dict):
                if 'best_model' in model:
                    xgb_model = model['best_model']
                elif 'booster' in model:
                    xgb_model = model['booster']
                elif 'model' in model:
                    xgb_model = model['model']
                else:
                    print(f"   [INFO] {model_name}: Chaves disponíveis: {list(model.keys())}")
                    raise ValueError(f"Não foi possível encontrar o modelo no dicionário")
            else:
                xgb_model = model
            
            dtest = xgb.DMatrix(X_test_xgb)
            probs = xgb_model.predict(dtest)
            
            if probs.max() > 1.0 or probs.min() < 0.0:
                probs = 1 / (1 + np.exp(-probs))
            
            preds = (probs >= 0.5).astype(int)
            
            predictions[model_name] = preds
            probabilities[model_name] = probs
        
        print(f"   [OK] {model_name}: {len(preds):,} predições")
        print(f"      Prob range: [{probs.min():.4f}, {probs.max():.4f}]\n")
        
    except Exception as e:
        print(f"   [ERRO] Erro ao fazer predições com {model_name}: {e}")
        import traceback
        print(f"      Detalhes: {traceback.format_exc()}")

print(f"\nPredições geradas para {len(predictions)} modelos!")

## Calcular Métricas para Todos os Modelos

In [ ]:
print("Calculando métricas...\n")

results = []

for model_name in predictions.keys():
    y_pred = predictions[model_name]
    y_prob = probabilities[model_name]
    
    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
    mcc = matthews_corrcoef(y_test, y_pred)
    aucpr = average_precision_score(y_test, y_prob)
    
    # Matriz de confusão
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    results.append({
        'Modelo': model_name,
        'Accuracy': accuracy,
        'AUCPR': aucpr,
        'ROC-AUC': roc_auc,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'MCC': mcc,
        'TN': tn,
        'FP': fp,
        'FN': fn,
        'TP': tp
    })

# Criar DataFrame com resultados
results_df = pd.DataFrame(results)

# Ordenar por AUCPR (descendente)
results_df = results_df.sort_values('AUCPR', ascending=False).reset_index(drop=True)

print("Métricas calculadas!\n")
print("=" * 120)
print("RESULTADOS COMPARATIVOS")
print("=" * 120)
print(results_df[['Modelo', 'AUCPR', 'Precision', 'Recall', 'F1-Score', 'MCC']].to_string(index=False))
print("=" * 120)

## 7. Classification Reports Detalhados

In [ ]:
print("\n" + "=" * 100)
print("CLASSIFICATION REPORTS")
print("=" * 100)

for model_name in predictions.keys():
    print(f"\n{'-' * 100}")
    print(f"{model_name}")
    print(f"{'-' * 100}")
    
    y_pred = predictions[model_name]
    
    # Classification report
    report = classification_report(y_test, y_pred, 
                                   target_names=['No Rain', 'Rain'],
                                   digits=4)
    print(report)
    
    # Resumo adicional
    aucpr = results_df[results_df['Modelo'] == model_name]['AUCPR'].values[0]
    roc_auc = results_df[results_df['Modelo'] == model_name]['ROC-AUC'].values[0]
    mcc = results_df[results_df['Modelo'] == model_name]['MCC'].values[0]
    
    print(f"AUCPR (Average Precision): {aucpr:.4f}")
    print(f"ROC-AUC: {roc_auc:.4f}")
    print(f"Matthews Correlation Coefficient: {mcc:.4f}")

print(f"\n{'=' * 100}")

## 8. Visualizar Matrizes de Confusão

In [ ]:
print("Gerando matrizes de confusão...\n")

n_models = len(predictions)

# FIGURA 1: MATRIZES NORMALIZADAS POR LINHA (RECALL)
fig, axes = plt.subplots(2, 3, figsize=(9, 6))
axes = axes.ravel()

for idx, model_name in enumerate(predictions.keys()):
    y_pred = predictions[model_name]
    cm = confusion_matrix(y_test, y_pred)
    
    cm_normalized_row = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    annot_labels = np.empty_like(cm, dtype=object)
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            annot_labels[i, j] = f'{cm[i, j]}\n({cm_normalized_row[i, j]*100:.1f}%)'
    
    sns.heatmap(cm_normalized_row, annot=annot_labels, fmt='', cmap='Blues', 
                xticklabels=['No Rain', 'Rain'],
                yticklabels=['No Rain', 'Rain'],
                ax=axes[idx],
                cbar_kws={'label': 'Proportion'},
                annot_kws={'fontsize': 10, 'fontweight': 'bold'},
                vmin=0, vmax=1)
    
    aucpr = results_df[results_df['Modelo'] == model_name]['AUCPR'].values[0]
    f1 = results_df[results_df['Modelo'] == model_name]['F1-Score'].values[0]

    axes[idx].set_title(f'{model_name}\nAUCPR: {aucpr:.4f}\nF1-Score: {f1:.4f}', 
                        fontsize=13, fontweight='bold', pad=10)
    axes[idx].set_xlabel('Predicted', fontsize=11, fontweight='bold')
    axes[idx].set_ylabel('Actual', fontsize=11, fontweight='bold')

if n_models < 6:
    fig.delaxes(axes[5])

plt.tight_layout()
plt.savefig(base_path / 'notebooks' / 'confusion_matrices_normalized_row.png', dpi=300, bbox_inches='tight')
print("Gráfico 1 salvo: confusion_matrices_normalized_row.png")
plt.show()

# FIGURA 2: MATRIZES NORMALIZADAS POR COLUNA (PRECISION)
fig, axes = plt.subplots(2, 3, figsize=(9, 6))
axes = axes.ravel()

for idx, model_name in enumerate(predictions.keys()):
    y_pred = predictions[model_name]
    cm = confusion_matrix(y_test, y_pred)
    
    cm_normalized_col = cm.astype('float') / cm.sum(axis=0)[np.newaxis, :]
    
    annot_labels = np.empty_like(cm, dtype=object)
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            annot_labels[i, j] = f'{cm[i, j]}\n({cm_normalized_col[i, j]*100:.1f}%)'
    
    sns.heatmap(cm_normalized_col, annot=annot_labels, fmt='', cmap='Greens', 
                xticklabels=['No Rain', 'Rain'],
                yticklabels=['No Rain', 'Rain'],
                ax=axes[idx],
                cbar_kws={'label': 'Proportion'},
                annot_kws={'fontsize': 10, 'fontweight': 'bold'},
                vmin=0, vmax=1)
    
    aucpr = results_df[results_df['Modelo'] == model_name]['AUCPR'].values[0]
    f1 = results_df[results_df['Modelo'] == model_name]['F1-Score'].values[0]
    
    axes[idx].set_title(f'{model_name}\nAUCPR: {aucpr:.4f}\nF1-Score: {f1:.4f}', 
                        fontsize=13, fontweight='bold', pad=10)
    axes[idx].set_xlabel('Predicted', fontsize=11, fontweight='bold')
    axes[idx].set_ylabel('Actual', fontsize=11, fontweight='bold')

if n_models < 6:
    fig.delaxes(axes[5])

plt.suptitle('Matrizes de Confusão - Normalizado por Coluna (Precision)', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(base_path / 'notebooks' / 'confusion_matrices_normalized_col.png', dpi=300, bbox_inches='tight')
print("Gráfico 2 salvo: confusion_matrices_normalized_col.png")
plt.show()

print("\n" + "="*80)
print("Todas as matrizes de confusão foram geradas!")
print("   2 figuras salvas:")
print("      1. confusion_matrices_normalized_row.png (normalizado por linha - Blues)")
print("      2. confusion_matrices_normalized_col.png (normalizado por coluna - Greens)")
print("="*80)

## 9. Comparação Visual de Métricas

In [ ]:
# Preparar dados para visualização
metrics_to_plot = ['AUCPR', 'Precision', 'Recall', 'F1-Score', 'MCC', 'ROC-AUC']

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for idx, metric in enumerate(metrics_to_plot):
    plot_data = results_df.sort_values(metric, ascending=True)
    
    bars = axes[idx].barh(plot_data['Modelo'], plot_data[metric])
    
    colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(plot_data)))
    for bar, color in zip(bars, colors):
        bar.set_color(color)
    
    for i, (bar, value) in enumerate(zip(bars, plot_data[metric])):
        axes[idx].text(value + 0.01, bar.get_y() + bar.get_height()/2, 
                      f'{value:.4f}', 
                      va='center', fontsize=9)
    
    axes[idx].set_xlabel(metric, fontsize=11, fontweight='bold')
    axes[idx].set_xlim(0, 1.1)
    axes[idx].grid(axis='x', alpha=0.3)
    axes[idx].set_title(f'Comparação: {metric}', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig(base_path / 'notebooks' / 'metrics_comparison.png', dpi=300, bbox_inches='tight')
print("Gráfico salvo: metrics_comparison.png")
plt.show()

## 10. Análise de Trade-offs: Precision vs Recall

In [ ]:
# Scatter plot: Precision vs Recall
plt.figure(figsize=(10, 8))

colors = plt.cm.Set2(np.arange(len(results_df)))

for idx, row in results_df.iterrows():
    plt.scatter(row['Recall'], row['Precision'], 
               s=500, alpha=0.6, color=colors[idx],
               edgecolors='black', linewidth=2)
    
    plt.annotate(row['Modelo'], 
                xy=(row['Recall'], row['Precision']),
                xytext=(10, 5), textcoords='offset points',
                fontsize=10, fontweight='bold')

plt.xlabel('Recall (Sensibilidade)', fontsize=13, fontweight='bold')
plt.ylabel('Precision', fontsize=13, fontweight='bold')
plt.title('Trade-off: Precision vs Recall\n(Maior = Melhor para ambos)', 
         fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xlim(0.45, 0.85)
plt.ylim(0.45, 0.85)

plt.axhline(y=0.5, color='red', linestyle='--', alpha=0.3, label='Baseline (0.5)')
plt.axvline(x=0.5, color='red', linestyle='--', alpha=0.3)
plt.legend()

plt.tight_layout()
plt.savefig(base_path / 'notebooks' / 'precision_recall_tradeoff.png', dpi=300, bbox_inches='tight')
print("Gráfico salvo: precision_recall_tradeoff.png")
plt.show()

## 11. Ranking Final dos Modelos

In [ ]:
print("\n" + "=" * 100)
print("RANKING FINAL DOS MODELOS (por AUCPR)")
print("=" * 100)

for idx, row in results_df.iterrows():
    position = ['1º', '2º', '3º', '4º', '5º'][idx]
    print(f"\n{position} Lugar: {row['Modelo']}")
    print(f"   - AUCPR: {row['AUCPR']:.4f}")
    print(f"   - ROC-AUC: {row['ROC-AUC']:.4f}")
    print(f"   - Precision: {row['Precision']:.4f}")
    print(f"   - Recall: {row['Recall']:.4f}")
    print(f"   - F1-Score: {row['F1-Score']:.4f}")
    print(f"   - MCC: {row['MCC']:.4f}")

print("\n" + "=" * 100)

## 12. Salvar Resultados em CSV

In [ ]:
# Salvar resultados completos
output_path = base_path / 'notebooks' / 'model_comparison_results.csv'
results_df.to_csv(output_path, index=False)
print(f"Resultados salvos em: {output_path}")

# Salvar também uma versão formatada
output_path_formatted = base_path / 'notebooks' / 'model_comparison_formatted.txt'
with open(output_path_formatted, 'w', encoding='utf-8') as f:
    f.write("=" * 100 + "\n")
    f.write("COMPARAÇÃO DE MODELOS - RESULTADOS FINAIS\n")
    f.write("=" * 100 + "\n\n")
    f.write(results_df.to_string(index=False))
    f.write("\n\n" + "=" * 100 + "\n")

print(f"Resultados formatados salvos em: {output_path_formatted}")